In [ ]:
!pip install -q beautifulsoup4
!pip install -q langchain
!pip install -q huggingface_hub
!pip install -q sentence_transformers
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install -q unstructured
!pip install -q pdf2image
!pip install -q pdfminer
!pip install -q rapidocr-onnxruntime
!pip install -q pypdfium2
!pip install -q weaviate-client
!pip install -q langchainhub
!pip install -q ray
!pip install -q openai
!pip install -q asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 5.2 MB/s eta 0:00

In [ ]:
import os
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["ANYSCALE_API_KEY"] = ""
WEAVIATE_API_KEY = os.environ["WEAVIATE_API_KEY"]
WEAVIATE_URL = ""
OPENAI_API_BASE=""
OPENAI_API_KEY=""

In [ ]:
import os
import requests
import numpy as np
#import openai
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader,UnstructuredPDFLoader, PyPDFium2Loader, TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS,Weaviate
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import Anyscale

def clean(s):
  # line = s
  line = s.replace('\r', ' ')
  line = line.replace('\n', ' ')

  return line


# - choose the embeddings Model
model_name = "thenlper/gte-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


loader = PyPDFium2Loader("/content/sample_data/MS-CS-Handbook-2022-2023Publish-7-9.pdf")
# loader = TextLoader("/content/sample_data/op7.txt")
documents = loader.load()

# documents = [clean(x) for x in documents]

fin_docs = []

for i in documents:
  temp = i
  temp.page_content = clean(temp.page_content)
  fin_docs.append(temp)



# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function=len
)
docs = text_splitter.split_documents(fin_docs)

In [ ]:
docs[0]

'into the M.S. program have been 153 or 63 percentile verbal, 163 or 88 percentiles  quantitative, and 4.0 analytical. However, admission decisions are made based on the entire   application packet. We do not require specific subject GRE scores. The ASU institution code is   4007. If a department code is required, use 000 for GRE.  English Proficiency - The University requires all international applicants from a country whose   native language is not English to provide the Test of English as a Foreign Language (TOEFL) or   the International English Language Testing System (IETLS) scores. CSE uses an average score   of 575 (paper-based) or 90 (internet-based) for TOEFL, or 7 for IETLS, or 65 for Pearson, or 115   for Duolingo for admission. Please note that your application will not be processed until the   university receives official English Proficiency scores, which are valid for two years from   the start date of the degree program. Exemption from the English Proficiency requirement

In [ ]:
#Look at sample embeddings
#sample_embedding = np.array(hf.embed_query(docs[0].page_content))
#print("Sample embedding of a document chunk: ", sample_embedding)
#print("Size of the embedding: ", sample_embedding.shape)

#create sample embeddings for the entire db
db = Weaviate.from_documents(docs, hf, weaviate_url=WEAVIATE_URL, by_text=False)


In [ ]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY)  # Replace w/ your Weaviate instance API key

# Instantiate the client with the auth config
client = weaviate.Client(
    url=WEAVIATE_URL,  # Replace w/ your endpoint
    auth_client_secret=auth_config
)

In [ ]:
import json
query = "What is the GRE requirment for MS?"
query_embedding = hf.embed_query(query)

response = (
    client.query
    .get("LangChain_bef45a57f1624e1f9ea3d3c7b6fe9b5d",["text"])
    .with_hybrid(
        query=query,
        vector=query_embedding
    )
    .with_limit(5)
        .do()
)

In [ ]:
temp = response['data']['Get']['LangChain_bef45a57f1624e1f9ea3d3c7b6fe9b5d']


In [ ]:
docs = [x['text'] for x in temp]

In [ ]:
docs

['into the M.S. program have been 153 or 63 percentile verbal, 163 or 88 percentiles  quantitative, and 4.0 analytical. However, admission decisions are made based on the entire   application packet. We do not require specific subject GRE scores. The ASU institution code is   4007. If a department code is required, use 000 for GRE.  English Proficiency - The University requires all international applicants from a country whose   native language is not English to provide the Test of English as a Foreign Language (TOEFL) or   the International English Language Testing System (IETLS) scores. CSE uses an average score   of 575 (paper-based) or 90 (internet-based) for TOEFL, or 7 for IETLS, or 65 for Pearson, or 115   for Duolingo for admission. Please note that your application will not be processed until the   university receives official English Proficiency scores, which are valid for two years from   the start date of the degree program. Exemption from the English Proficiency requiremen

In [ ]:
import os


In [ ]:
from langchain.chat_models import ChatAnyscale

chats = {
    model: ChatAnyscale(model_name=model, temperature=0.2)
    for model in ChatAnyscale.get_available_models()
}


In [ ]:
chats = {'meta-llama/Llama-2-70b-chat-hf':chats['meta-llama/Llama-2-70b-chat-hf']}

In [ ]:
# context = [x.page_content for x in relevant_documents]
fin_cont = " ".join(docs)

In [ ]:
fin_cont

'into the M.S. program have been 153 or 63 percentile verbal, 163 or 88 percentiles  quantitative, and 4.0 analytical. However, admission decisions are made based on the entire   application packet. We do not require specific subject GRE scores. The ASU institution code is   4007. If a department code is required, use 000 for GRE.  English Proficiency - The University requires all international applicants from a country whose   native language is not English to provide the Test of English as a Foreign Language (TOEFL) or   the International English Language Testing System (IETLS) scores. CSE uses an average score   of 575 (paper-based) or 90 (internet-based) for TOEFL, or 7 for IETLS, or 65 for Pearson, or 115   for Duolingo for admission. Please note that your application will not be processed until the   university receives official English Proficiency scores, which are valid for two years from   the start date of the degree program. Exemption from the English Proficiency requirement

In [ ]:
query

'What is the GRE requirment for MS?'

In [ ]:
import asyncio

from langchain.schema import SystemMessage, HumanMessage


messages = [
    SystemMessage(
        content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know."
    ),
    HumanMessage(
        content= "Context: "+ fin_cont + "Question: "+ query + " Answer:"
    ),
    ]

async def get_msgs():
    tasks = [
        chat.apredict_messages(messages)
        for chat in chats.values()
    ]
    responses = await asyncio.gather(*tasks)
    return dict(zip(chats.keys(), responses))

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
response_dict = asyncio.run(get_msgs())

## The GRE requirement for the MS program in Computer Science at Arizona State University is a minimum score of 153 or 63 percentile verbal, 163 or 88 percentiles quantitative, and 4.0 analytical. However, it's important to note that admission decisions are made based on the entire application packet, and the department does not require specific subject GRE scores.

---
